In [ ]:
import requests

import json

import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
api_key = "cbcffe5648e04b07afa203548241202"  # API key obtained from https://www.worldweatheronline.com/
location_list = [
    "02459"
]  # list of strings containg US Zipcode, UK Postcode, Canada Postalcode, IP address, Latitude/Longitude (decimal degree) or city name
start_date = "01-JAN-2021"  # date when desired scraping period starts; preferred date format: 'dd-MMM-yyyy', e.g., '01-JAN-2020'
end_date = "31-DEC-2022"  # date when desired scraping period ends; preferred date format: 'dd-MMM-yyyy
frequency = 24  # frequency between observations; possible values 1 (1 hour), 3 (3 hours), 6 (6 hours), 12 (12 hours (day/night)) or 24 (daily averages)

In [1]:
import datetime as dt
from projects.weather.wwo_data_reader import DataReader


d=DataReader.create('wwo','/Users/lindseygulden/dev/leg-up/config/weather/newton_weather.yml')


In [5]:
d.data_renaming_dict.keys() #['hourly_variables']

dict_keys(['daily_variables', 'hourly_variables', 'astronomy_variables'])

In [2]:
d.prep_query()
d.get_data()
d.postprocess_data()
d.write_data()

2020-01-01 00:00:00
True


TypeError: expand_weather_data() missing 1 required positional argument: 'data_config_path'

In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
from projects.utils.data import convert_to_datetime

In [ ]:
convert_to_datetime(dt.date(2020,1,3))

In [ ]:
convert_to_datetime(pd.Timestamp(year=2017, month=1, day=1, hour=12))

In [ ]:
convert_to_datetime(np.datetime64(12013103,'s'))

In [ ]:
d=dt.datetime(2020,1,1)
d.to_pydatetime()

In [ ]:
payload = {
    "key": api_key,
    "date": start_date,
    "enddate": end_date,
    "q": location_list[0],
}

In [ ]:
dict([[1,2],[2,3]])

In [ ]:
response = requests.get(
    f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&date={start_date}&enddate={end_date}&q={location_list[0]}&tp=1&format=json&includelocation=yes"
)

In [ ]:
response.content

In [ ]:
daily_weather_vars=['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF',
       'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', ]
astronomy_vars=['sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination']
hourly_vars=['time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex']
extract_weather_desc=False
if 'weatherDesc' in hourly_vars:
       hourly_vars=[x for x in hourly_vars if x != 'weatherDesc']
       extract_weather_desc=True

In [ ]:
mydict={'locations': ['02459'],
'api_key': "cbcffe5648e04b07afa203548241202"  ,
'entry_point': "https://api.worldweatheronline.com/premium/v1/past-weather.ashx"  ,
'start_date' : "01-JAN-2020" ,
'end_date' : "31-DEC-2022" ,
'timeout_seconds': 30,
'frequency' : 24 ,
'daily_weather_variables':['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF',
       'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', ]}

In [ ]:
from schema import Schema, And, Use, Optional, SchemaError
import yaml
import validators
schema = Schema(
        {
            'locations': And(list, lambda x: all([isinstance(xx,str) for x in xx])),
            'api_key': And(str),
            Optional('entry_point',default='https://api.worldweatheronline.com/premium/v1/past-weather.ashx'): And(str,lambda x : validators.url(x)),
            'end_date': And(str,lambda x: len(x)==11),
            'start_date': And(str,lambda x: len(x)==11),
            Optional('timeout_seconds',default=30): And(int, lambda t: t>0),
            Optional('frequency',default=1): And(lambda f : f in [1, 3, 6, 12, 24],int),
            'daily_weather_variables': And(list, lambda x: all([xx in ['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF','avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex'] for xx in x]) ),
            'astronomy_variables': And(list, lambda x: all([xx in ['sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'] for xx in x]) ),
            'hourly_variables': And(list, lambda x: all([xx in ['time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex'] for xx in x]) )
        }
    )

In [ ]:
df

In [ ]:


weather=json.loads(response.content.decode('utf-8'))['data']['weather']

weather_list=[]

for i in range(len(weather)):
    weather_dict={k : weather[i][k] for k in tuple(daily_weather_vars)}
    weather_dict.update({k : weather[i]['astronomy'][0][k] for k in tuple(astronomy_vars)})
    for h in range(len(weather[i]['hourly'])):
        weather_dict.update({k : weather[i]['hourly'][h][k] for k in tuple(hourly_vars)})
        if extract_weather_desc:
            weather_dict.update({'weatherDesc':weather[i]['hourly'][h]['weatherDesc'][0]['value']})
        weather_list.append(weather_dict.copy())
weather_df=pd.DataFrame(weather_list)



In [ ]:
weather_df.date.value_counts()

In [ ]:
with open("/Users/lindseygulden/dev/leg-up/data/newton_tp1.json", "wb") as fd:
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)

In [ ]:
# df = pd.read_json("/Users/lindseygulden/dev/leg-up/data/newton_tp24.json")
df = pd.read_json("/Users/lindseygulden/dev/leg-up/data/newton_tp1.json")
df

In [ ]:
h_list=[]
w_list=[]
n_entries=len(df.loc['weather',]['data'])
for d in range(n_entries):
    w_list.append(df.loc['weather',]['data'][d])

all_weather_df=pd.DataFrame(w_list)
forecast_vars=['date','maxtempF','mintempF','avgtempF','totalSnow_cm','sunHour','uvIndex']


In [ ]:
n_hours=all_weather_df.loc[0,'hourly']

In [ ]:
import yaml
with open('/Users/lindseygulden/dev/leg-up/projects/weather/config/data_description.yml','r') as f:
    try:
        tmp=yaml.safe_load(f)    
    except yaml.YAMLError as e:
        print(e)


In [ ]:
import yaml
import datetime as dt
with open('../../test.yml','w') as f:
    yaml.dump({'d':{'first':dt.datetime(2001,1,1)}},f,default_flow_style=False)

In [ ]:

with open('../../config.yml', 'w') as outfile:
    yaml.dump(hourly[0], outfile, default_flow_style=False)
    


In [ ]:
daily_weather_vars=['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF',
       'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', ]
astronomy_vars={'sunrise':'sunrise_local_time', 'sunset':'sunset_local_time', 'moonrise':'moonrise_local_time', 
                'moonset':'moonset_local_time', 'moon_phase':'moon_phase', 'moon_illumination':'moon_illumination'}
hourly_vars=['time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex']


In [ ]:
all_weather_df.loc[0,'astronomy']

In [ ]:

h_list=[]
w_list=[]
for d in range(35):
    w_list.append(df.loc['weather',]['data'][d])

all_weather_df=pd.DataFrame(w_list)
forecast_vars=['date','maxtempC', 'mintempC', 'avgtempC','maxtempF','mintempF','avgtempF','totalSnow_cm','sunHour','uvIndex']
forecast_df=all_weather_df[forecast_vars]


for d in range(35):
    hourly=all_weather_df.loc[d,'hourly']
    astronomy=all_weather_df.loc[d,'astronomy'][0]
    today_date=all_weather_df.loc[d,'date']
    for hour in hourly:
        # add daily variables to hourly variables
        for v in astronomy_vars:
            hour[v]=astronomy[v]
        for v in forecast_vars:
            hour[v]=all_weather_df.loc[d,v]
        h_list.append(hour)
hourly_df=pd.DataFrame(h_list)
hourly_df=hourly_df[daily_weather_vars+hourly_vars]

In [ ]:
hourly_df

In [ ]:
config = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up/config/weather/newton_weather.yml"
)

In [ ]:
hourly_df.date

In [ ]:
all_weather_df.loc[0, forecast_vars]

In [ ]:
all_weather_df.loc[0, "hourly"]

In [ ]:
hourly_df.loc

In [ ]:
for i in hourly_df.columns.values:
    print(f"   - '{i}'")

In [ ]:
import sys

print(sys.path)

import sys
print(sys.path)

In [ ]:
from projects.utils.io import yaml_to_dict

In [ ]:
tmp = pd.DataFrame(w_list)
len(tmp.loc[0, "hourly"])